In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1566222863010_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-3>

In [2]:
%%configure -f
{"driverMemory": "10240M", "executorMemory": "5120M", "executorCores": 4, "conf":{"spark.pyspark.python": "python","livy.server.session.timeout-check": "false" , "livy.server.session.timeout": 7200000 }}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1566222863010_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1566222863010_0003,pyspark,idle,Link,Link,
4,application_1566222863010_0005,pyspark,idle,Link,Link,✔


In [3]:
from collections import defaultdict
import json
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

In [4]:

df_1=spark.read.json("s3://nsegmentlogs-v3/*/").filter( col("propertyId").isNotNull() & 
                                                             col("userId").isNotNull()
                                                    )     


In [5]:
df_na=spark.read.json("s3://nsegmentlogs-v3-na/*/").filter( col("propertyId").isNotNull()
                                                    )     


In [ ]:
'''
from functools import reduce
from pyspark.sql import DataFrame

dfs = [df_1,df_na]
df = reduce(DataFrame.unionAll, dfs)
'''

In [6]:
#df_1.count()

40546

In [7]:
#df_na.count()

2477

In [8]:
#merge both datasets
df_1.registerTempTable("df_tbl")
df_na.registerTempTable("df_na_tbl")


In [22]:
#int(time.mktime(time.strptime('2017-07-4', '%Y-%m-%d')))

#int(time.mktime(time.strptime(eventTimestamp, '%Y%m%d%H%M%S')))

#2019-06-30T03:10:38.493Z

#1504137600
        

1499126400

In [9]:
import time
def ttime(val):
    return int(time.mktime(time.strptime(val, '%Y%m%d%H%M%S')))

In [10]:
spark.udf.register("ttime", ttime)


<function ttime at 0x7f1c1e7ddb90>

In [11]:
df_union=spark.sql('''
select userId as USER_ID, propertyId as ITEM_ID, replace(propertyCountry,'Canada','CA') as COUNTRY, ttime(eventTimestamp) as TIMESTAMP from df_tbl
UNION ALL
select NVL(NVL(contactId.Id, userId),1) as USER_ID, propertyId as ITEM_ID, replace(propertyCountry,'Canada','CA') as COUNTRY, ttime(eventTimestamp) as TIMESTAMP from df_na_tbl
''')

In [12]:
df_union.printSchema()

root
 |-- USER_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)

In [13]:
df_union.registerTempTable("df_union_tbl")

In [14]:
spark.catalog.cacheTable("df_union_tbl")

In [13]:
#df_union.count()
# July 10 - 36604
# July 16 - 37832

37832

In [15]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd
#g_input="mongodb://10.0.1.70"

g_input="mongodb://3.210.155.32"

client = MongoClient(g_input,27017)


db = client.backend_production


In [16]:
import pandas as pd
df_pd = pd.DataFrame(columns=['propertyId'])
l_cnt=0

In [17]:
for i in df_union.select('ITEM_ID').distinct().collect():
    cnt= db.properties.find( {'_id': ObjectId(i[0]), 'status': 'active'}).count()
    #cnt= db.properties.find( {'_id': ObjectId(i[0])}).count()
    if cnt == 0:
        l_cnt=l_cnt+1
        df_pd.loc[l_cnt] = [i[0]]

In [18]:
df_pd.shape

(7231, 1)

In [19]:
spark_df = sqlContext.createDataFrame(df_pd)

In [20]:
spark_df.registerTempTable("spark_df_tbl")

In [21]:
df_active=spark.sql('''
select * from df_union_tbl a where 1=1 and not exists (select 1 from spark_df_tbl b where a.ITEM_ID=b.propertyId)

''')

In [22]:
df_active.registerTempTable("df_active_tbl")

In [50]:
spark.sql('''
select country, count(*) from df_active_tbl group by 1
''').show()

+-------+--------+
|country|count(1)|
+-------+--------+
|   null|     182|
|     CA|    2026|
|     US|    8095|
+-------+--------+

In [23]:
spark.sql('''
select country, count(*) from df_active_tbl group by 1
''').show()

+-------+--------+
|country|count(1)|
+-------+--------+
|   null|     197|
|     CA|    3265|
|     US|    9461|
+-------+--------+

In [52]:
spark.sql('''
select country, count(distinct item_id, user_id) from df_active_tbl group by 1
''').show()

+-------+--------------------------------+
|country|count(DISTINCT item_id, user_id)|
+-------+--------------------------------+
|   null|                              33|
|     CA|                             704|
|     US|                            2057|
+-------+--------------------------------+

In [24]:
spark.sql('''
select country, count(distinct item_id, user_id) from df_active_tbl group by 1
''').show()

+-------+--------------------------------+
|country|count(DISTINCT item_id, user_id)|
+-------+--------------------------------+
|   null|                              48|
|     CA|                            1275|
|     US|                            2823|
+-------+--------------------------------+

In [53]:
spark.sql('''
select country, count(distinct item_id, user_id, timestamp) from df_active_tbl group by 1
''').show()

+-------+-------------------------------------------+
|country|count(DISTINCT item_id, user_id, timestamp)|
+-------+-------------------------------------------+
|   null|                                         55|
|     CA|                                       1208|
|     US|                                       4089|
+-------+-------------------------------------------+

In [25]:
spark.sql('''
select country, count(distinct item_id, user_id, timestamp) from df_active_tbl group by 1
''').show()

+-------+-------------------------------------------+
|country|count(DISTINCT item_id, user_id, timestamp)|
+-------+-------------------------------------------+
|   null|                                         68|
|     CA|                                       2444|
|     US|                                       5492|
+-------+-------------------------------------------+

In [26]:
df_active_filter=spark.sql('''
select distinct user_id, item_id, country, timestamp from df_active_tbl
''').count()

In [27]:
df_active_filter=df_active.select('user_id', 'item_id', 'country', 'timestamp').distinct()



In [28]:
df_active_filter.registerTempTable("df_active_filter_tbl")


In [60]:
df_active_filter.count()

5352

In [29]:
df_active_filter.repartition(1) \
        .write.format("com.databricks.spark.csv") \
        .partitionBy('country')\
        .mode("overwrite") \
        .option("header","true")\
        .save("s3://nestready-amazon-personalize/events-active")


In [30]:
spark.catalog.uncacheTable("df_union_tbl")

In [ ]:
### Filter old items from metadata so that coldstart items remain less than 80K

In [90]:
df_items_events= spark.read.load("s3://nestready-amazon-personalize/events-active/country=US/", 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

In [91]:
df_items_events.count()

4089

In [40]:
df_items_metdata = spark.read.load("s3://nestready-amazon-personalize/items/country=US/", 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

In [41]:
df_items_metdata.count()

2361107

In [92]:
df_items_events.registerTempTable("df_items_events_tbl")

df_items_metdata.registerTempTable("df_items_metdata_tbl")


In [93]:
df_items_events.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- timestamp: integer (nullable = true)

In [94]:
df_items_metdata.printSchema()

root
 |-- ITEM_ID: string (nullable = true)
 |-- PRICE: integer (nullable = true)
 |-- BEDS: integer (nullable = true)
 |-- CITY: string (nullable = true)
 |-- POSTAL: string (nullable = true)
 |-- CREATED_AT: integer (nullable = true)

In [25]:
def reverse_unix_format(p_dt):
    import datetime
    dt = datetime.datetime.fromtimestamp(p_dt).strftime('%Y-%m-%d')
    return dt

In [107]:
print (int(time.mktime(time.strptime('20190530101010', '%Y%m%d%H%M%S'))))

1559211010

In [100]:
df=spark.sql('''
select * from df_items_metdata_tbl a
where 1=1
and (exists (select 1 from df_items_events_tbl b where b.item_id=a.item_id)
OR CREATED_AT > 1559174400
)
''')

In [106]:
df.repartition(1) \
        .write.format("com.databricks.spark.csv") \
        .mode("overwrite") \
        .option("header","true")\
        .save("s3://nestready-amazon-personalize/items-filtered/US/")
